# 파이썬 크롤링 (beautifulsoup, selenium 활용)

* **bs4를 활용한 크롤링**   
    다음 [블로그](https://beomi.github.io/2017/01/20/HowToMakeWebCrawler/)를 참고하면 좋을 것 같습니다!

* **selenium을 사용한 크롤링**   
    beautifulsoup은 사용자 행동을 특정해서 데이터를 가져올 수 없다. 사용자의 행동을 동적으로 추가하려면 **selenium**이 필요하다.    
    Selenium은 webdriver라는 것을 통해 디바이스에 설치된 브라우저들을 제어할 수 있다.

* **selenium 및 웹 드라이버 설치**   
    1. selenium 설치 --> $ pip install selenium   
    2. 웹 드라이버 설치 --> Chrome WebDriver 설치 (https://sites.google.com/a/chromium.org/chromedriver/downloads) 후 로컬에 저장, 압축 해제
    3. 크롬드라이버의 위치(경로)를 나중에 Selenium 객체를 생성할 때 지정해준다.

## **0. Selenium 간단한 활용 예시**

불러오기

In [1]:
from selenium import webdriver

selenium의 webdriver로 크롬 브라우저를 실행

In [2]:
driver = webdriver.Chrome('/home/hufs/chromedriver') #  크롬드라이버 위치 지정

Naver 에 접속한다.

In [3]:
driver.get("http://www.naver.com")

검색창에 커서를 올린다.

In [4]:
sch = driver.find_element_by_xpath('//*[@id="query"]')

검색어를 입력한다.

In [5]:
sch.send_keys("Agilesoda")

검색을 실행한다.

In [6]:
sch.submit()

검색이 제대로 됐는지 확인한다.

In [7]:
assert "No results found." not in driver.page_source

브라우저를 종료한다.

In [8]:
driver.close()

## 1. 뉴스 기사 수집하기
### 목표 : 네이버 뉴스 기사 데이터 수집   
(http://news.naver.com/main/ranking/popularWeek.nhn?mid=etc&sid1=111)

* 기사 본문과 제목 수집에는 newspaper 모듈[(참고)](http://excelsior-cjh.tistory.com/entry/06-Newspaper-%EB%AA%A8%EB%93%88%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-%EB%89%B4%EC%8A%A4-%EA%B8%B0%EC%82%AC-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0)을 활용하였습니다.    
* '주간 클릭' 상위 30개를 가져오는 코드입니다.   

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin
from newspaper import Article
import re
import time
import pandas as pd

driver = webdriver.Chrome('/home/hufs/chromedriver') 

In [10]:
# 날짜와 카테고리를 선택

article_date = '1006'   # 1006, 1013, 1020, 1027, ... , 1229
article_ctg = [100,101,102,103,104,105]  # 정치(100), 경제(101), 사회(102), 생활/문화(103), 세계(104), IT/과학(105)


## 카테고리별로 자동 반복

for ctg in article_ctg:
    driver.get("http://news.naver.com/main/ranking/popularWeek.nhn?rankingType=popular_week&sectionId={}&date=2017{}".format(
        ctg, article_date))   # 카테고리와 날짜에 따라서 url 주소가 달라집니다.  # 주간 클릭 페이지로 이동    
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml") 
    raw_lists = soup.find_all('dt')

    url = []  
    ## 카테고리 내에서 30개 기사마다 url 주소 다름
    for i in range(len(raw_lists[:-1])):
        child = raw_lists[i].findChildren() 
        url.append(child[0]['href'])


    base_address = "http://news.naver.com"
    for i in range(len(url)):      # 0~ 29 총 30개 기사마다 url 다름
        url[i] = urljoin(base_address, url[i])   

    data_list = []   
    for i in range(0,30): # 1~30위 기사 추출 ---- By.  Newspaper module ----
        ID = i
        titles = []
        body_content = []  
        driver.get(url[i])      
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")  # 파싱할 문서를 전달합니다.
        # 기사 제목 
        temp = Article(url[i], language='ko')
        temp.download()
        temp.parse()
        print("Crawled: " + temp.title)
        titles.append(temp.title)    # rbind!
        # 기사 본문 
        body_content.append(temp.text)
        # file write
        data = {'ID':ID, 'Category': ctg, 'Week' : article_date, 'Title':titles[0], 'Contents': body_content[0]}  # 기사 1개 write
        data_list.append(data) # 기사 1개씩 덧붙이기
    
        df = pd.DataFrame(data_list, columns=['ID', 'Category','Week','Title','Contents'])
        df.to_csv('Article{}_{}.txt'.format(ctg, article_date), sep=',', encoding='UTF-8')  ## 카테고리별로 text 파일이 저장됩니다!
        
driver.close()

Crawled: 北 "서울을 위험에 빠뜨리지 않는 군사적 선택안은 없다"
Crawled: 北 개성공단 19곳 몰래 가동…정부 "불법행위"
Crawled: [박수찬의 軍] 한반도 위기는 미국 군수산업 호황을 부른다
Crawled: "불과 1년만에"···朴·최순실, 권력 정점에서 밑바닥 추락
Crawled: 가속 붙은 국정원 수사…MB 사법처리까지 갈까
Crawled: 北, '개성공단 의류공장 가동설'에 "공장 더 힘차게 돌아갈 것"
Crawled: '北항공기 저공침투 막아라'…한미 軍, 첫 단거리방공훈련
Crawled: 文, 北 동향 즉시 보고 받아…軍 '비상 태세'
Crawled: '군부대 맞아?'...철원 총기사고 현장 가보니, 뻥~뚫린 軍 작전지역 '여전'
Crawled: '공영방송 블랙리스트 의혹' 김재철 등 추석 후 '줄소환'
Crawled: 文대통령, 교통방송 일일통신원 변신…"졸음운전 안전띠는 휴식"
Crawled: 4년 만에 지방권력 바꾸는 내년 지방선거…여야에서 누가 뛰나
Crawled: 열악한 환경에 승조원 70%, 잠수함 안 탄다
Crawled: 한국형 3축 체계 완성할 무기 종류·도입 시기는
Crawled: 엄현성 해군총장, 이지스함서 北 탄도미사일 작전 대응태세 점검
Crawled: 사업단 해체 불구…4천t급 '핵잠 원자로' 기본설계 마쳤다
Crawled: '최순실 게이트' 수사 착수 1년···검찰의 칼은 2R 돌입
Crawled: “도비탄 아닌 직격탄 가능성” 파장 커지는 철원 총기 사고
Crawled: 北 "美 초강도 제재, 효과 기대하는 것은 어리석은 짓"
Crawled: [단독] 방사청 "중고 치누크 헬기 도입 과정 문제 있었다"
Crawled: "원유 의존성 없애자"…제재 속 '에너지 자립' 고심하는 北
Crawled: 文대통령, 트럼프 대통령에 위로전…"비극적 총격사건 규탄"
Crawled: "역대급 수소탄…" 北 위협발언에 南 유행어 등장
Crawled: "盧·DJ정권, 적폐 없었겠느냐" vs "MB 포토라인에 설 것"
Cra